<a href="https://colab.research.google.com/github/drakax1/Google-Colab/blob/main/Scraper_Anibis_Autos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
url = "https://www.anibis.ch/fr/c/automobiles-voitures-de-tourisme?pi="

ads_links = []

for page in range(1,2):
  driver.get(url+str(page))
  links = driver.find_elements_by_xpath("//a[contains(@class,'sc-AxirZ bdqkCI sc-1gbeqqm-0 fiNCt')]")

  for i in links:
    ads_links.append(i.get_attribute('href'))
    print(f"{ads_links.index(i.get_attribute('href'))+1}/{len(ads_links)}: {i.get_attribute('href')}")

In [ ]:
f=open('liens_voiture.txt','w')
ads_links=map(lambda x:x+'\n', ads_links)
f.writelines(ads_links)
f.close()

In [ ]:
anibis = pd.DataFrame(columns = ["Marque","Modèle","Année","Kilomètres","Prix","Lien"])

for i in ads_links:
  print(f"Processing {ads_links.index(i)+1}/{len(ads_links)}: {i}")
  try:
    driver.get(i)

    if driver.find_elements_by_xpath("//*[contains(@href,'https://kredit.financescout24.ch/')]"):

      j = driver.find_elements_by_xpath("//*[@id='root']/div/div[5]/div/div[1]/div[3]/div[1]/div[1]/div[2]/div/ul/li[2]/div[2]/span")[0]
      brand = j.text

      k = driver.find_elements_by_xpath("//*[@id='root']/div/div[5]/div/div[1]/div[3]/div[1]/div[1]/div[2]/div/ul/li[3]/div[2]/span")[0]
      model = k.text

      l = driver.find_elements_by_xpath("//*[@id='root']/div/div[5]/div/div[1]/div[3]/div[1]/div[1]/div[2]/div/ul/li[5]/div[2]/span")[0]
      year = l.text

      m = driver.find_elements_by_xpath("//*[@id='root']/div/div[5]/div/div[1]/div[3]/div[1]/div[1]/div[2]/div/ul/li[4]/div[2]/span")[0]
      km = m.text
      km = km.replace(" km","")

      try:
        n = driver.find_elements_by_xpath("//*[@id='root']/div/div[5]/div/div[1]/div[1]/div[4]/div[1]")[0]
        price = n.text
        price = price.replace(" / Prix à discuter","")

      except:
        n = driver.find_elements_by_xpath("//*[@id='root']/div/div[5]/div/div[1]/div[1]/div[3]/div[1]")[0]
        price = n.text
        price = price.replace(" / Prix à discuter","")

      anibis.loc[len(anibis)] = [brand,model,year,km,price,i]

    else:
      j = driver.find_elements_by_xpath("//*[@id='root']/div/div[5]/div/div[1]/div[3]/div[1]/div[1]/div[2]/div/ul/li[1]/div[2]/span")[0]
      brand = j.text

      k = driver.find_elements_by_xpath("//*[@id='root']/div/div[5]/div/div[1]/div[3]/div[1]/div[1]/div[2]/div/ul/li[2]/div[2]/span")[0]
      model = k.text

      l = driver.find_elements_by_xpath("//*[@id='root']/div/div[5]/div/div[1]/div[3]/div[1]/div[1]/div[2]/div/ul/li[4]/div[2]/span")[0]
      year = l.text

      m = driver.find_elements_by_xpath("//*[@id='root']/div/div[5]/div/div[1]/div[3]/div[1]/div[1]/div[2]/div/ul/li[3]/div[2]/span")[0]
      km = m.text
      km = km.replace(" km","")

      try:
        n = driver.find_elements_by_xpath("//*[@id='root']/div/div[5]/div/div[1]/div[1]/div[4]/div")[0]
        price = n.text
        price = price.replace(" / Prix à discuter","")

      except:
        n = driver.find_elements_by_xpath("//*[@id='root']/div/div[5]/div/div[1]/div[1]/div[3]/div")[0]
        price = n.text
        price = price.replace(" / Prix à discuter","")

      anibis.loc[len(anibis)] = [brand,model,year,km,price,i]

  except Exception as e:
      print(f"{e}: {i}")

display(anibis)

In [ ]:
anibis1 = anibis.copy()

In [ ]:
anibis1[["Kilomètres","Prix"]] = anibis1[["Kilomètres","Prix"]].apply(lambda x: x.str.replace("Gratuit|Prix sur demande","0").str.replace("CHF|'|\.|–|\s",""))
anibis1[["Année","Kilomètres","Prix"]] = anibis1[["Année","Kilomètres","Prix"]].astype(int)
display(anibis1.head(40).sort_values(by=["Année","Kilomètres","Prix"], ascending=[False,False,False]))

In [ ]:
anibis1.query("Marque == ['Audi','BMW'] and Année >= 2012").sort_values(["Année","Kilomètres","Prix"], ascending=[False,True,True])

In [ ]:
anibis1[["Marque","Kilomètres"]].query("Marque == ['Audi','BMW']").groupby("Marque").agg(['mean', 'count'])

In [ ]:
anibis1.query("Prix <= 5000 and Prix != [0,88] and Kilomètres <= 200000").sort_values("Prix", ascending=True)